# deeper multilayer perception with dropout

In [1]:
# load package and mnist
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('../MNIST_data/', one_hot=True)
%matplotlib inline  
print ("PACKAGES LOADED")

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
PACKAGES LOADED


## define model

In [2]:
# network topologies
n_input = 784  # 28 x 28
n_hidden_1 = 512  # 1st layer num features
n_hidden_2 = 512  # 2st layer num features
n_hidden_3 = 256  # 3st layer num features
n_classes = 10 

# input and output
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])
dropout_keep_prob = tf.placeholder("float")

# variable
stddev = 0.05
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1], stddev=stddev)),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], stddev=stddev)),
    'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3], stddev=stddev)),
    'out': tf.Variable(tf.random_normal([n_hidden_3, n_classes], stddev=stddev))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'b3': tf.Variable(tf.random_normal([n_hidden_3])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

print("network ready")

Instructions for updating:
Colocations handled automatically by placer.
network ready


## MLP as a function

In [3]:
def multilayer_perception(_x, _weights, _biases, _keep_prob):
    layer_1 = tf.nn.relu(tf.add(tf.matmul(_x, _weights['h1']), _biases['b1']))
    layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, _weights['h2']), _biases['b2']))
    layer_3 = tf.nn.relu(tf.add(tf.matmul(layer_2, _weights['h3']), _biases['b3']))
    dropout_layer = tf.nn.dropout(layer_3, _keep_prob)
    return(tf.add(tf.matmul(dropout_layer, _weights['out']), _biases['out']))

## define function

In [4]:
# prediction and loss
# pred = tf.nn.softmax(multilayer_perception(x, weights, biases, dropout_keep_prob))
# loss = tf.reduce_mean(-tf.reduce_sum(y * tf.log(pred), reduction_indices=1)) 

pred = multilayer_perception(x, weights, biases, dropout_keep_prob)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))

# optimizer
learning_rate = 0.01
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

# accuracy
corr = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(corr, "float"))

# init
init = tf.global_variables_initializer()

print("function ready")

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

function ready


## run

In [5]:
# parameter
training_epoches = 20
batch_size = 100
display_step = 4

# launch the graph
sess = tf.Session()
sess.run(init)

# train
for epoch in range(training_epoches):
    avg_cost = 0
    n_batch = int(mnist.train.num_examples / batch_size)
    for i in range(n_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        train_feed = {x: batch_xs, y: batch_ys, dropout_keep_prob: 0.6}
        sess.run(optimizer, feed_dict = train_feed)
        feed = {x: batch_xs, y: batch_ys, dropout_keep_prob: 1.0}
        avg_cost += sess.run(loss, feed_dict = feed)
    avg_cost = avg_cost / batch_size
    
    # display
    if (epoch + 1) % display_step == 0:
        train_acc = sess.run(accuracy, feed_dict = feed)
        test_feed = {x: mnist.test.images, y: mnist.test.labels, dropout_keep_prob: 1.0}
        test_acc = sess.run(accuracy, feed_dict = test_feed)
        print("Epoch: %03d/%03d  " "loss: %.5f  " "train accuracy: %.3f  " "test accuracy: %.3f"
             % (epoch + 1, training_epoches, avg_cost, train_acc, test_acc))
        

Epoch: 004/020  loss: 0.59721  train accuracy: 0.970  test accuracy: 0.954
Epoch: 008/020  loss: 0.51564  train accuracy: 0.990  test accuracy: 0.958
Epoch: 012/020  loss: 0.45088  train accuracy: 0.980  test accuracy: 0.962
Epoch: 016/020  loss: 0.44641  train accuracy: 0.970  test accuracy: 0.955
Epoch: 020/020  loss: 0.39985  train accuracy: 0.970  test accuracy: 0.959


In [6]:
# close
sess.close()